[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/tp/01_aprendizaje_supervisado/tp01-enunciado.ipynb)

# Trabajo Práctico -  Aprendizaje supervisado
### Clasificación de expresiones genómicas

<span style="color: red;">**Fecha de entrega: Jueves 01 de mayo del 2025 - hasta las 17:00hs.**
 
<span style="color: red;">**Fecha de entrega intermedia: Jueves 17 de Abril del 2025 - hasta las 17:00hs.**
</span>

## Introducción

En el mundo actual, distintas disciplinas científicas empiezan, cada vez más, a interactuar con el fin de potenciar sus descubrimientos. En este caso dos grupos de investigación de [CONICET](https://www.conicet.gov.ar/) se embarcan en la combinación entre biología y informática para abordar la detección temprana y el pronóstico preciso de enfermedades como el cáncer. Este proyecto combina las tecnologías de secuenciación de nueva generación ([_NGS_](https://es.wikipedia.org/wiki/Secuenciaci%C3%B3n_paralela_masiva), por sus siglas en inglés) con la potencia de la inteligencia artificial. El enfoque se centra en un dataset único que abarca mediciones de [_ARN_](https://es.wikipedia.org/wiki/ARN_mensajero) de 200 [_genes_](https://es.wikipedia.org/wiki/Gen), recopiladas de pacientes con lesiones [_pre-tumorales_](https://en.wikipedia.org/wiki/Hyperplasia). Este conjunto de datos se convierte en una valiosa fuente de información para entender cómo las células en estado de hiperplasia pueden evolucionar hacia [_tumores malignos_](https://en.wikipedia.org/wiki/Neoplasm), una transformación que ha desconcertado a la ciencia durante décadas.

La hiperplasia, es un fenómeno en el que las células experimentan un crecimiento anormal y descontrolado, es un punto de partida crucial en nuestro análisis. ¿Cómo y por qué algunas células que experimentan hiperplasia se convierten en células cancerosas, mientras que otras no? Esta pregunta es el corazón de nuestra investigación. Para responderla se realizo un estudio donde se obtuvieron muestras de distintos tipos de hiperplasias de pacientes con antecedentes familiares y lesiones pre tumorales. Este grupo de pacientes, o cohorte, fue monitoreado periodicamente durante los siguientes 5 años buscando indicios de neoplasias o nuevas hiperplasias más agresivas. Con las muestras obtenidas en este estudio se realizo un [_biobanco_](https://en.wikipedia.org/wiki/Biobank) con las mediciones que habitualmente se hacen en la construccion de este tipo de [_plataformas_](https://xena.ucsc.edu/). Cada muestra fue etiquetada como **_buen pronostico_**, si no hubo indicios de nuevas hiperplasias o similares; contrariamente se etiquetaron como de **_mal pronostico_** si hubo una recaida.

Este trabajo se concentra en un panel de genes, especificamente en la expresion de 200 genes que se creen tienen un papel crucial en la transformacion tumoral y su etiqueta correspondiente.

En concreto:

Tendrán un archivo `.csv` en donde se almacenan:
  - una matriz de datos `X` de $500$ filas en donde cada fila $x^{(i)}$ representa un vector de $200$ características de cada instancia. Es decir, $\textbf{x}^{(i)} = x_1^{(i)}, \dots, x_{200}^{(i)}$ con $i$ entre $1$ y $500$.
  - una columna llamada `target` que representa un vector de $500$ posiciones con dos posibles valores: `True` (ó 1, es decir, tiene buen pronostico) y `False` (ó 0, tiene mal pronostico).

Los datos están en esta [carpeta](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

Por otra parte, tendrán disponibles un conjunto de instancias sin etiquetas, que utilizaremos para comprobar la calidad de sus resultados (ver Ejercicio 5). 

**Recomendamos fuertemente leer primero todo el enunciado del trabajo antes de empezar a trabajar sobre el problema propuesto.**

---

### Sobre el informe

Para este trabajo deberán entregar, además del código de las pruebas y experimentos que realicen, un informe en el que deberan seleccionar, para cada apartado, sus resultados acompañado de un texto que explique, reflexione, justifique y conluya dicho contenido. 

Cada ejercicio indica el largo máximo del texto que se puede incluir. Los gráficos no están contados en dicho espacio. 
Cada gráfico incluido debe contar con:
  
  - nombres de los ejes,
  - título,
  - leyenda autocontenida,
  - debe ser referenciado desde el texto, ya que su inclusión se da porque aporta a la discusión del trabajo.

**El informe no puede superar un máximo de 8 carillas (contando gráficos) o 4 hojas más carátula.** Tamaño de la letra: estandár de latex (10pt). No se corregirán trabajos que no cumplan con esta consigna.

---

In [2]:
import numpy as np
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold,RandomizedSearchCV,ParameterGrid
from sklearn.metrics import accuracy_score, precision_recall_curve, auc, roc_auc_score,average_precision_score

In [56]:
df=pd.read_csv("data.csv")
df.head()

,QqgU,IJsJ,YlJA,tFLR,MYkJ,Cado,cmdU,AUYy,mVEY,zSZq,...,cgNV,naPw,EOre,sMSU,TihR,sbWq,aWBQ,fGxK,JWOq,target
0,-0.753155,-0.304399,0.307146,-1.377240,-1.975242,-0.052628,0.430082,-0.659100,4.897377,0.754531,...,0.975926,-0.593156,0.315798,0.546475,1.128513,7.654107,0.563439,0.322854,-1.754513,0
1,0.660977,-1.374880,-0.039899,-0.157981,-0.263226,-0.035127,1.583745,-0.143602,12.945694,-0.347768,...,-0.274560,0.205678,-0.296301,-1.004013,7.090070,0.182824,0.113722,1.323722,0.710734,0
2,1.906825,-0.308091,1.115546,0.093203,-1.384119,0.616543,0.384448,-0.817921,-7.673495,1.028706,...,-0.369924,-0.678204,1.564510,0.314275,-4.246162,-1.941577,-0.566316,-0.261689,-1.715006,1
3,3.767180,-0.495569,-0.654660,0.101866,0.644159,1.978273,-1.087526,0.925767,10.392570,0.203025,...,0.252199,-0.200540,1.043750,-0.039129,-2.898566,-5.231800,-0.476618,-0.300148,0.315923,0
4,3.540548,0.544672,-1.820594,0.068094,0.844530,-0.516157,0.561619,1.634488,1.220771,-0.344691,...,-2.380954,1.532346,-1.594870,-1.809599,-0.445860,-0.169324,1.876894,-0.043769,1.529304,0


In [58]:
df[df['target']==0]

,QqgU,IJsJ,YlJA,tFLR,MYkJ,Cado,cmdU,AUYy,mVEY,zSZq,...,cgNV,naPw,EOre,sMSU,TihR,sbWq,aWBQ,fGxK,JWOq,target
0,-0.753155,-0.304399,0.307146,-1.377240,-1.975242,-0.052628,0.430082,-0.659100,4.897377,0.754531,...,0.975926,-0.593156,0.315798,0.546475,1.128513,7.654107,0.563439,0.322854,-1.754513,0
1,0.660977,-1.374880,-0.039899,-0.157981,-0.263226,-0.035127,1.583745,-0.143602,12.945694,-0.347768,...,-0.274560,0.205678,-0.296301,-1.004013,7.090070,0.182824,0.113722,1.323722,0.710734,0
3,3.767180,-0.495569,-0.654660,0.101866,0.644159,1.978273,-1.087526,0.925767,10.392570,0.203025,...,0.252199,-0.200540,1.043750,-0.039129,-2.898566,-5.231800,-0.476618,-0.300148,0.315923,0
4,3.540548,0.544672,-1.820594,0.068094,0.844530,-0.516157,0.561619,1.634488,1.220771,-0.344691,...,-2.380954,1.532346,-1.594870,-1.809599,-0.445860,-0.169324,1.876894,-0.043769,1.529304,0
5,4.764378,1.136220,-1.008677,0.079692,-1.299948,0.405142,-0.788473,-0.564579,-12.028644,-0.994163,...,0.549378,1.027353,1.361510,0.737509,4.491953,-5.600168,-0.308669,-1.408245,-1.250049,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0.297857,-0.632737,-0.944735,0.441495,0.273719,0.807160,-0.920723,-0.593019,2.435399,-0.409558,...,1.929605,-1.317757,1.293989,-1.784298,-0.790963,-3.190602,-1.597291,-0.562582,0.340068,0
496,-1.045521,-0.875682,-0.829544,-1.494002,0.436961,-0.494770,-0.219712,-0.387546,-0.751816,0.051520,...,-1.157861,-2.462821,-0.571710,-0.239654,4.685438,1.120013,-0.723162,-0.155328,1.200592,0
497,-3.385825,-1.500261,-0.923048,0.002346,-1.189872,-1.471672,0.678777,-1.268629,-13.890491,1.880261,...,0.880166,1.157510,-0.135622,-0.866339,-3.999636,5.705475,0.426428,0.007213,1.047125,0
498,-5.068169,-0.783885,-0.819872,-0.906944,0.210135,0.629747,0.802087,0.333887,2.750179,-0.200211,...,-0.548944,0.258575,-0.522897,-0.139874,0.307522,2.683021,-0.336143,0.031017,-0.294661,0


## Ejercicio 1 

### Separación de datos

Contarán con una cantidad limitada de datos, por lo cual es importante tomar una buena decisión en el momento de empezar a utilizarlos. 

Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. ¿Qué consideraciones tuvieron en cuenta para realizar esta división?

**Importante**: en este punto no está permitido dividir la base de datos utilizando la función `train_test_split` de sklearn. Deben decidir e implementar la separación.

In [55]:
def train_test_split(df,target_name,eval_proportion= 0.2,random_seed = 0):

    n_total = len(df)
    n_eval = int( eval_proportion* n_total)  
    df_c1 = df[df[target_name] == 1]
    df_c2 = df[df[target_name] == 0]
    df_c1 = df_c1.sample(frac=1, random_state=random_seed).reset_index(drop=True)
    df_c2 = df_c2.sample(frac=1, random_state=random_seed).reset_index(drop=True)
    n_eval_c1 = int(eval_proportion * len(df_c1))
    n_eval_c2 = n_eval - n_eval_c1   
    df_eval_c1 = df_c1.iloc[:n_eval_c1]
    df_eval_c2 = df_c2.iloc[:n_eval_c2]
    df_dev_c1 = df_c1.iloc[n_eval_c1:]
    df_dev_c2 = df_c2.iloc[n_eval_c2:]
    df_eval = pd.concat([df_eval_c1, df_eval_c2]).sample(frac=1, random_state=42).reset_index(drop=True)
    df_dev = pd.concat([df_dev_c1, df_dev_c2]).sample(frac=1, random_state=42).reset_index(drop=True)
    X_test = df_eval.drop(columns=[target_name])
    y_test = df_eval[target_name]
    X_train = df_dev.drop(columns=[target_name])
    y_train = df_dev[target_name]

    return X_test,y_test,X_train,y_train



In [22]:
X_test,y_test,X_train,y_train = train_test_split(df,'target')

## Ejercicio 2


### Construcción de modelos

Para este punto, la tarea consiste en construir y evaluar modelos de tipo **árbol de decisión**. Además, obtener una **estimación realista de la performance** de los mismos.

1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

1. Estimar la performance del modelo utilizando _K-fold cross validation_ con `K=5`, con las métricas _Accuracy_, _Area Under the Precision-Recall Curve (AUPRC)_, y _Area Under the Receiver Operating Characteristic Curve (AUCROC)_. 

   En esta oportunidad se va a pedir además de calcular las métricas para cada fold por separado y su promedio, que hagan el cálculo del score global (como vimos en clase), sólo para los folds de validación.
   
   Reportar el resultado en una tabla similar a:

      <table>
      <thead>
      <tr>
      <th align="center">Permutación</th>
      <th>Accuracy (training)</th>
      <th>Accuracy (validación)</th>
      <th>AUPRC (training)</th>
      <th>AUPRC (validación)</th>
      <th>AUC ROC (training)</th>
      <th>AUC ROC (validación)</th>
      </tr>
      </thead>
      <tbody>
      <tr>
      <td align="center">1</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">2</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">3</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">4</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">5</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">Promedios</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <td align="center">Global</td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      </tr>
      </tbody>
      </table>    
  
   **Importante**: de acá en más sólamente utilizaremos el score promedio cuando hagamos _K-fold cross-validation_.
 
1. Explorar las siguientes combinaciones de parámetros para  árboles de decisión (siguiendo con $k-fold$ con $k=5$) utilizando [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) de _scikit learn_. No está permitido utilizar `GridSearchCV` en este ejercicio.

   <table>
   <thead>
   <tr>
   <th align="center">Altura máxima</th>
   <th align="center">Criterio de corte</th>
   <th>Accuracy (training)</th>
   <th>Accuracy (validación)</th>
   </tr>
   </thead>
   <tbody><tr>
   <td align="center">3</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">3</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   </tbody></table>

1. ¿Qué conclusiones se pueden sacar de estas tablas?  

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold,RandomizedSearchCV,ParameterGrid
from sklearn.metrics import accuracy_score, precision_recall_curve, auc, roc_auc_score,average_precision_score

In [ ]:
def Ejercicio2(X,y,criterion,grid = None,depth=3,scores = None,random_seed=0, n_splits= 5, shuffle = True):

    if grid:
        grid=list(ParameterGrid(grid))
        for param in grid:
            tree=tree_model=DecisionTreeClassifier(**param)
            skf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_seed)
            scores=cross_validate(
                tree_model,
                X,
                y,
                cv=skf,
                scoring="accuracy",
                return_train_score=True
                )
            print(f"Parametros {param} con test accuracy: {scores['test_score'].mean()}")
            print(f"Parametros {param} con train accuracy: {scores['train_score'].mean()}")
            print("\n")
        exit
    else:
        tree = DecisionTreeClassifier(
            max_depth=depth,
            criterion=criterion,
            random_state=random_seed)
        skf = StratifiedKFold(
            n_splits=n_splits,
            shuffle=shuffle,
            random_state=random_seed)
        score = cross_validate(
            tree,   
            X,
            y,
            cv=skf,
            scoring=scores,
            return_train_score=True)
        for metrics in scores:
            test_metric = score[f'test_{metrics}']
            train_metric = score[f'train_{metrics}']
            for i in range(len(test_metric)):
                print(f"El valor de la metrica test_{metrics} en la iteracion {i} es {test_metric[i]}")
            print(f"El promedio para test_{metrics} es {test_metric.mean()}")
            print("\n")
            for i in range(len(train_metric)):
                print(f"El valor de la metrica train_{metrics} en la iteracion {i} es {train_metric[i]}")
            print(f"El promedio para train_{metrics} es {train_metric.mean()}")
            print("\n")


#### Primer parte

In [46]:
scoring = {
    'accuracy': 'accuracy',
    'auprc': 'average_precision',
    'aucroc': 'roc_auc'
}

In [ ]:
Ejercicio2(X_train,y_train,criterion='gini',depth=3,scores=scoring)

#### Segunda parte

In [ ]:
param_grid={"max_depth":[3,5,None],"criterion":["gini","entropy"]}

In [54]:
Ejercicio2(X_train,y_train,grid=param_grid,criterion='gini',depth=3,scores=scoring)

Parametros {'criterion': 'gini', 'max_depth': 3} con test accuracy: 0.655
Parametros {'criterion': 'gini', 'max_depth': 3} con train accuracy: 0.828125


Parametros {'criterion': 'gini', 'max_depth': 5} con test accuracy: 0.6325
Parametros {'criterion': 'gini', 'max_depth': 5} con train accuracy: 0.9237500000000001


Parametros {'criterion': 'gini', 'max_depth': None} con test accuracy: 0.605
Parametros {'criterion': 'gini', 'max_depth': None} con train accuracy: 1.0


Parametros {'criterion': 'entropy', 'max_depth': 3} con test accuracy: 0.6825
Parametros {'criterion': 'entropy', 'max_depth': 3} con train accuracy: 0.7875


Parametros {'criterion': 'entropy', 'max_depth': 5} con test accuracy: 0.64
Parametros {'criterion': 'entropy', 'max_depth': 5} con train accuracy: 0.9075000000000001


Parametros {'criterion': 'entropy', 'max_depth': None} con test accuracy: 0.6024999999999999
Parametros {'criterion': 'entropy', 'max_depth': None} con train accuracy: 1.0




## Ejercicio 3

### Comparación de algoritmos 

Se pide explorar distintas combinaciones de algoritmos de aprendizaje con diferentes configuraciones con el objetivo de **encontrar el mejor modelo** de cada familia de buscar la performance óptima. Para este ejercicio realizar una experimentación utilizando [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Como métrica de performance usar AUCROC resultante de 5-fold cross-validation. 

Hiperparámetros_: Revisar la documentación de cada uno de los hiperparámetros para entender qué posibles hiperparámetros impacten de manera positiva en la construcción del algoritmo.

Documentación extra sobre [`Tuning hyper-parameters`](https://scikit-learn.org/stable/modules/grid_search.html), leer hasta 3.2.2.

1. Algoritmos a probar: 
  - Árboles de decisión. Mínimo 4 hiperparámetros.
  - KNN (k-vecinos más cercanos). Mínimo 3 hiperparámetros.
  - SVM (Support vector machine). Mínimo 2 hiperparámetros.

Detallar los hiperparámetros elegidos para cada algoritmo y explicar la razón del espacio de búsqueda considerado para cada uno de estos, ¿cuántas iteraciones usaron?. A su vez, reportar la performance asociada de aquellos que consideren relevantes (al menos la mejor combinación para cada algoritmo). 

2. Compare los resultados obtenidos en el ejercicio anterior con los siguientes modelos con sus hiperparámetros default. 

  - LDA (Linear discriminant analysis)
  - Naïve Bayes

¿Qué resultados obtuvo? ¿Qué hiperparámetros podrían ser relevantes explorar en estos modelos? ¿Por qué?

3. ¿Cuál fue el mejor modelo y con qué configuración? Explicar por qué creería que dio mejor (recordando qué hace cada algoritmo y con qué tipo de datos están trabajando).

In [ ]:
from scipy.stats import randint

Primero lo hacemos para arboles de decision

In [13]:
#Primero lo hacemos para arboles de decision
param_arboles={"criterion":["gini","entropy"],"max_depth":randint(1,10),
               'splitter': ['best', 'random'],"min_samples_leaf":randint(1,10)}
n_iter=30
k=5
tree_model=DecisionTreeClassifier(random_state=42)
random_search=RandomizedSearchCV(tree_model,param_distributions=param_arboles,n_iter=n_iter,cv=k,scoring="roc_auc",
                                 random_state=42)
random_search.fit(X_dev,y_dev)
print(f"Los mejores hiperparametros son : {random_search.best_params_}")
print(f"El score promedio del mejor modelo es :{random_search.best_score_}")
#Me fijo el score para los datos de evaluacion
mejores_parametros=random_search.best_params_
mejor_modelo=DecisionTreeClassifier(**mejores_parametros)
mejor_modelo.fit(X_dev,y_dev)
y_pred=mejor_modelo.predict(X_eval)
y_proba = mejor_modelo.predict_proba(X_eval)[:, 1]
print(f"El score sobre los datos de evaluacion es : {roc_auc_score(y_eval,y_proba)}")

Los mejores hiperparametros son : {'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 8, 'splitter': 'best'}
El score promedio del mejor modelo es :0.6371971861471861
El score sobre los datos de evaluacion es : 0.5423809523809524


Ahora para K-NN


In [14]:
from sklearn.neighbors import KNeighborsClassifier
param_knn={'n_neighbors': randint(3, 20),
    'weights': ['uniform', 'distance'],
    'p': [1, 2] }
knn=KNeighborsClassifier()
random_search=RandomizedSearchCV(knn,param_distributions=param_knn,n_iter=n_iter,cv=k,scoring="roc_auc")
random_search.fit(X_dev,y_dev)
print(f"Los mejores hiperparametros son : {random_search.best_params_}")
print(f"El score promedio del mejor modelo es :{random_search.best_score_}")
#Me fijo el score para los datos de evaluacion
mejores_parametros=random_search.best_params_
mejor_modelo=KNeighborsClassifier(**mejores_parametros)
mejor_modelo.fit(X_dev,y_dev)
y_pred=mejor_modelo.predict(X_eval)
y_proba = mejor_modelo.predict_proba(X_eval)[:, 1]
print(f"El score sobre los datos de evaluacion es : {roc_auc_score(y_eval,y_proba)}")

Los mejores hiperparametros son : {'n_neighbors': 8, 'p': 1, 'weights': 'distance'}
El score promedio del mejor modelo es :0.8517910173160175
El score sobre los datos de evaluacion es : 0.8657142857142858


Vamos con SVM

In [28]:
from sklearn.svm import SVC
param_svm={"kernel":["linear","poly","rbf"],"C":randint(10,50),"degree":randint(1,5),"max_iter":randint(1000,2000)}
svm=SVC()
grid_svm=RandomizedSearchCV(svm,param_distributions=param_svm,n_iter=n_iter,cv=k,scoring="roc_auc")
grid_svm.fit(X_dev,y_dev)
print(grid_svm.best_params_)
print(grid_svm.best_score_)
best_param=grid_svm.best_params_
#Ahora me fijo que onda el score en los datos de evaluacion
best_svm=SVC(**best_param,probability=True)
best_svm.fit(X_dev,y_dev)
y_prob=best_svm.decision_function(X_eval)
print(roc_auc_score(y_eval,y_prob)) #da mas alto?


C:\Users\54117\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1003).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\54117\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1003).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\54117\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1003).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\54117\AppData\Local\Packages\PythonSoftwareFou

{'C': 26, 'degree': 4, 'kernel': 'rbf', 'max_iter': 1477}
0.89171341991342
0.9133333333333333


Ahora probamos con LDA

In [16]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA_model=LinearDiscriminantAnalysis()
LDA_model.fit(X_dev, y_dev)
y_proba_dev = LDA_model.predict_proba(X_dev)[:, 1]
y_proba_eval=LDA_model.predict_proba(X_eval)[:,1]
# Calcular el AUROC
auc_dev = roc_auc_score(y_dev, y_proba_dev)
auc_eval=roc_auc_score(y_eval,y_proba_eval)

print("AUROC_DEV:", auc_dev)
print("AUROC_EVAL", auc_eval) #overfittea?

AUROC_DEV: 0.9879143339553896
AUROC_EVAL 0.7680952380952382


Vamos con Naive-Bayes Gaussiano

In [25]:
from sklearn.naive_bayes import GaussianNB

gaussian_nb=GaussianNB()
gaussian_nb.fit(X_dev,y_dev)
y_pred_scores=gaussian_nb.predict_proba(X_eval)[:,1]
y_pred_scores_train=gaussian_nb.predict_proba(X_dev)[:,1]
print(f"La metrica AUC-ROC para Gaussian Naive-Bayes en los datos de desarrollo es :{roc_auc_score(y_eval,y_pred_scores)}")
print(f"La metrica AUC-ROC para Gaussian Naive-Bayes en los datos de entrenamiento es :{roc_auc_score(y_dev,y_pred_scores_train)}")


La metrica AUC-ROC para Gaussian Naive-Bayes en los datos de desarrollo es :0.7923809523809524
La metrica AUC-ROC para Gaussian Naive-Bayes en los datos de entrenamiento es :0.9582629817233923


## Ejercicio 4
### Diagnóstico Sesgo-Varianza.

<span style="color: red;">(no realizar hasta la clase _Sesgo y Varianza_)</span>

En este punto, se pide inspeccionar **tres** de sus mejores modelos encontrados hasta ahora de cada familia de modelos: la mejor configuración para el árbol de decisión y la mejor configuración para SVM. Para ello:

1. Graficar curvas de complejidad para cada modelo, variando la profundidad en el caso de árboles, y el hiperparámetro C en el caso de SVM. Diagnosticar cómo afectan al sesgo y a la varianza esos dos hiperparámetros.
2. Graficar curvas de aprendizaje para cada modelo pero ahora incluya LDA. En base a estas curvas, sacar conclusiones sobre si los algoritmos parecen haber alcanzado su límite, o bien si aumentar la cantidad de datos debería ayudar.
3. Construir un modelo **RandomForest** con 200 árboles. Explorar para qué sirve el hiperparámetro max_features y cómo afecta a la performance del algoritmo mediante una curva de complejidad. Explicar por qué creen que se dieron los resultados obtenidos. Por último, graficar una curva de aprendizaje sobre los parámetros elegidos para determinar si sería útil o no conseguir más datos.


**Atención**: Tener en cuenta que debemos seguir utilizando AUC ROC como métrica para estas curvas.


## Ejercicio 5: 
### Evaluación de performance

- La entrega del trabajo estará acompañada de una evaluación en la cual deberán poner a prueba su mejor modelo y sobre todo, su capacidad para estimar sus resultados. 

- Su tarea será estimar la performance (AUCROC) que tendrá su mejor modelo en datos de evaluación (X_held_out). 

- Para ello, deberán predecir las **probabilidades** de las distintas instancias con su modelo, enviarnos dichas probabilidades junto a una estimación con 4 decimales de cuál será el AUCROC resultante y calcularemos el resultado real. Consideraremos que el **mejor modelo será el que se encuentre más cerca del valor real que calcularemos luego de la entrega**.

- Recomendamos no perder de vista esta evaluación/competencia durante el desarrollo del TP, sobretodo en el momento de separar los datos en los primeros puntos. 

- Para que podamos evaluar la performance, junto con la entrega del informe, deberán enviar un archivo con el numero de grupo con dos digitos en formato csv con la columna `output` y el valor obtenido con 4 decimales (se subirá un ejemplo cuando se publiquen los datos de la competencia) y un valor esperado de AUCROC: `GG_y_pred_held_out_AUCROC`. 

    - Ej.: el grupo tres cree que obtuvo un valor de 0.7321 de AUCROC deberá submitear un archivo llamado: `03_y_pred_held_out_7321.csv`.

- Los datos podrán encontrarlos en este [link](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

- Las decisiones de este punto pueden desarrollarse hasta en una carilla, aunque con media debería alcanzar.


## Ejercicio 6: 
### Conclusiones

Escribir como mínimo en un párrafo, una conclusión del trabajo realizado, incluyendo problemas encontrados y 
aspectos no incluidos en el enunciado que hayan sido abordadas durante el desarrollo.

---
## Entregables
- Contarán con un esqueleto en formato Jupyter Notebook en donde podrán intercalar celdas para reportar y responder a los ítems de cada ejercicio. 
- Los entregrables serán
    - Un informe en formato .pdf (**digital**) que responda a los ítems de este enunciado respetando la cantidad de espacio máximo por cada ítem. Nombrarlo siguiendo el formato `GG_Nombre_de_grupo`
    - Adjuntar el notebook final en formatos .pdf e .ipynb. Es necesario que los resultados puedan reproducirse al ejecutar todas las celdas en orden (verificarlo haceindo: Kernel -> Restart and Run All). 
    - Las predicciones del *held out* del punto 5 en formato csv.
- Habŕa una entrega intermedia obligatoria que deberán hacer antes del 17 de abril de 2025 a las 17:00hs. Para esta entrega deberán enviar el código que resuelve los primeros 3 ejercicios. 
- La **fecha** y **hora límite** de entrega está determinada en el campus de la materia.
- El trabajo deberá elaborarse en grupos de 5 personas.
- Se podrán pedir pruebas de integridad y autoría; es decir, verificar que la salida solicitada es fruto del modelo presentado y que el modelo fue construido según lo requerido en este enunciado.
- La evaluación será grupal y se basará en la calidad del informe (presentación, claridad, prolijidad); la originalidad, practicidad y coherencia técnica de la solución; la corrección y solidez de las pruebas realizadas.


### Importante: sobre el uso de ChatGPT y grandes modelos de lenguaje

En este trabajo no estará explícitamente prohibido pero si fuertemente desaconsejado, consideramos a este trabajo práctico una importante herramienta de aprendizaje donde el uso de GPT puede ser perjudicial. En caso de usarlo se pide aclararlo en el informe y especificar cómo y en donde se utilizó. Así como expresar su opinión sobre la respuesta generada por el modelo pudiendo estar a favor o en contra de lo propuesto por este. Pueden adjuntar el link a la conversación con el modelo.

**Nota**: Agradecemos a [Martín García Sola](https://ar.linkedin.com/in/martin-e-garcia-sola) por la asistencia biológica en la confección de este Trabajo Práctico.